# PythonのAutoFeatを使った自動特徴量エンジニアリング （Automatic Feature Engineering）その3（特徴量選択だけ使う）

url: https://www.salesanalytics.co.jp/datascience/datascience029/

    第232話｜3タイプの特徴量エンジニアリング（feature engineering）基礎テクニック

url: https://www.salesanalytics.co.jp/column/no00232/#Wrapper_Method

    Pythonに幾つかの自動特徴量エンジニアリング（Automatic Feature Engineering）のためのパッケージがあります。

    その中の1つに「AutoFeat」というものがあります。回帰問題と分類問題で利用できます。

    AutoFeatは非常に簡単に使えます。
    ただ、完全な特徴量エンジニアリング（Feature Engineering）を実施してくれるわけではありません。
    例えば、欠測値補完や異常値処理、正規化処理などは実施しません。

    何をやるかと言うと……

    元の特徴量からたくさんの非線形の特徴量を生成し「特徴量プール」を作ります
    「特徴量プール」から有効な特徴量を選択します（特徴量選択）
    ……という感じです。

    前々回は分類問題で、前回は回帰問題でした。

    特徴量選択の機能だけ使いたい、という方もいるかもしれません。ということで今回は、特徴量選択の機能だけ使う場合のやり方について、簡単に説明します。

    【ちょっと理論的なお話し】AutoFeatの特徴量選択方法
    先ず、AutoFeatの特徴量選択方法について、簡単な解説をします。興味のない方は、飛ばしてください。

    先ほどの説明の繰り返しになりますが、ざっくり説明するとAutoFeatの特徴量エンジニアリング（Feature Engineering）は、次のようになっています。

    1.元の特徴量からたくさんの非線形の特徴量を生成し「特徴量プール」を作ります
    2.「特徴量プール」から有効な特徴量を選択します（特徴量選択）

    「特徴量プール」とは、元の特徴量と新たに生成した非線形の特徴量を合わせた特徴量の集合のことです。この「特徴量プール」に対し、特徴量選択を実施していきます。

    特徴量選択は、ざっくり次の2つのステップからなります。

    ・先ず、相関の高い特徴量を削る
    ・次に、L1正則化で特徴量を選択する

    相関の高い特徴量を削るとき、より単純な特徴量（元の特徴量に近い方）を残します。

    例えば、xとzという元の特徴量からx^2とz^2という特徴量を作ったとします。xとz^2の相関が高いときxの方を残しz^2の方を削ります。

    L1正則化で特徴量で選択するとき、Lasso LARS回帰モデルとL1正則化ロジスティック回帰モデルを活用したラッパー法（Wrapper Method）を使い、特徴量選択をしています。

    ・回帰問題に対しLasso LARS回帰モデル
    ・分類問題に対しL1正則化ロジスティック回帰モデル

    L1正則化で特徴量を選択するとき、「特徴量プール」から「初期セット」を選びます。
    そして、「特徴量プール」に残ったものを幾つかに分割し「特徴量のチャンク（塊）」を作ります。
    「初期セット」に各チャンクを追加し特徴量セットを作りモデルをフィットさせ、特徴量選択の検討を実施していきます。
    そういう意味でラッパー法（Wrapper Method）です。

    ラッパー法（Wrapper Method）に関しては以下の記事で簡単に解説していますので、参考にして頂ければと思います。

    第232話｜3タイプの特徴量エンジニアリング（feature engineering）基礎テクニック
    https://www.salesanalytics.co.jp/column/no00232/#Wrapper_Method

    ちなみに、「初期セット」は、すべての特徴量を使いL1正則化線形モデルを学習し、係数の絶対値の大きな特徴量を選んだものです。

    ロバスト性（頑健性）を高めるために、データのサンプリングなどを何度も何度も実施し、このようなことを何度も何度も繰り返し検討し、
    比較的いつも残る（選択される）特徴量を、最終的な特徴量として選択します。

    なぜでこのような面倒なことをするのか？　という疑問もあることでしょう。

    最初に実施する、「すべての特徴量を使いL1正則化線形モデルを学習し、係数の絶対値の大きな特徴量を選ぶ」、それだけでいいのではないか？　と思うかもしれません。

    しかし、それでは上手くいかないことが知られています。

    「すべての特徴量を使いL1正則化線形モデルを学習し、係数の絶対値の大きな特徴量を選ぶ」だけで上手くいく条件があります。それは、特徴量同士の相関が低い場合です。

    今回の場合ですと、元の特徴量から非線形な変換をして作っていることもあり、特徴量同士の相関が低いとは言い切れず、
    どちらかというと相関の高い特徴量の組み合わせが混じっている可能性が高いです。
    今回の例に限らず、どちらかというと相関の高い特徴量の組み合わせが混じっている可能性が高いため、
    「すべての特徴量を使いL1正則化線形モデルを学習し、係数の絶対値の大きな特徴量を選ぶ」という方法は多くの場合あまり適切とは言えません。

    気になる方は、以下を参考にして頂ければと思います。

    Agnostic feature selection
    https://hal.archives-ouvertes.fr/hal-02436824

    そのため、AutoFeatではこのようなアプローチで特徴量選択を実施しています。

## AutoFeatのFeature Selectorを使って見よう！
    今回は、意図的に特徴量（説明変数）を膨大にするために、Scikit-Learn（sklearn）を使い以下のようなサンプルデータセットを生成し、特徴量選択を実施しています。

    回帰問題用に生成するサンプルデータ
    ・サンプル数：1,100
    ・特徴量の数：1,000

    分類問題用に生成するサンプルデータ
    ・サンプル数：1,100
    ・特徴量の数：1,000
    ・目的変数のクラス数：2

## ライブラリーの読み込み
    先ず、必要なライブラリーを読み込みます。

In [ ]:
pip install autofeat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.0/305.0 kB 8.2 MB/s eta 0:00:00


In [ ]:
# ライブラリーの読み込み
import pandas as pd
from autofeat import FeatureSelector
from sklearn.datasets import make_regression, make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.metrics import r2_score,accuracy_score

## 回帰問題に対する特徴量選択例
    次に、サンプルデータセットを生成します。

    以下、コードです。

In [ ]:
# サンプルデータの生成
X, y = make_regression(n_samples=1100,   #サンプルの数
                       n_features=1000,  #特徴量の数
                       n_informative=10, #目的変数と関連性の強い特徴量の数
                       noise=10,         #ノイズの標準偏差
                       bias=0,           #バイアス（切片）
                       random_state=12)

X = pd.DataFrame(X)
y = pd.DataFrame(y)

In [ ]:
display(X)
display(y)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


0      0
1      0
2      0
3      0
4      0
      ..
564    0
565    0
566    0
567    0
568    1
Name: target, Length: 569, dtype: int64

    生成したデータセットを学習データとテストデータに分割します。

    以下、コードです。

In [ ]:
# データセットの分割（学習データとテストデータ）
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    train_size=0.5,
                                                    test_size=0.5,
                                                    random_state=42)

    学習データ
    特徴量：X_train
    目的変数：y_train
    
    テストデータ
    特徴量：X_test
    目的変数：y_test

    変数選択用のモデルを定義します。

    以下、コードです。

In [ ]:
# モデル定義
model = FeatureSelector(problem_type="regression",verbose=1)

    学習データを利用し特徴量選択を実施します。

    以下、コードです。

In [ ]:
# 特徴量選択（学習データ利用）
X_train_fsel = model.fit_transform(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)


[featsel] Scaling data...done.


KeyboardInterrupt: 

    元の特徴量からどのくらい減ったのかを見てみます。

    以下、コードです。

In [ ]:
print("number of features in X_train:",
      X_train.shape[1])                  #元の特徴量Xの数
print("number of features in X_train_fsel:",
      X_train_fsel.shape[1])             #選択された特徴量Xの数

# 1,000個あった特徴量が、23個に減っています。

    これで、モデルの予測精度が恐ろしく悪化したのでは、身も蓋もありません。

    テストデータで確認してみます。

    この自動選択した特徴量と同じ特徴量のテストデータを作ります。X_test_fselに格納します。

    以下、コードです。

In [ ]:
# テストデータの特徴量生成
X_test_fsel = model.transform(X_test)

Index(['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup',
       'Latitude', 'Longitude', 'MedInc**5', 'HouseAge/AveOccup',
       'log(MedInc)/AveOccup', 'Longitude**3/Latitude',
       'HouseAge**3*Population', 'AveBedrms**2*MedInc**3',
       'MedInc**2*log(AveOccup)', '1/(HouseAge*Population)',
       'log(Population)/AveBedrms', 'sqrt(Population)/AveOccup',
       'sqrt(HouseAge)*Latitude**3', 'sqrt(HouseAge)*log(AveRooms)',
       'Population/MedInc', 'AveRooms/AveBedrms', 'HouseAge*MedInc**3',
       '1/(AveOccup*MedInc)', 'AveOccup/Population', 'sqrt(HouseAge)*MedInc',
       'log(AveBedrms)/MedInc', '1/(AveBedrms*AveOccup)',
       'HouseAge**2/Population', '1/(AveBedrms*Latitude)',
       'Population**2/AveOccup', 'log(AveRooms)/AveRooms',
       'Latitude**3*log(MedInc)', 'HouseAge*log(AveBedrms)',
       'MedInc*sqrt(Population)', 'AveRooms*sqrt(Population)',
       'log(AveOccup)*log(MedInc)', 'sqrt(Population)/HouseAge',
       'sqrt(AveRooms)*Latitu

    元の特徴量のデータセット（X_trainとy_train）と新たな特徴量のデータセット（X_train_fselとy_train）で、線形回帰（重回帰）モデルを構築してみます。

    以下、コードです。

In [ ]:
print("number of features in X_train:",  #元の特徴量Xの数
      X_train.shape[1])
print("number of features in X_train_feature_creation:",
      X_train_feature_creation.shape[1]) #新しい特徴量Xの数

# 元の特徴量は30個でしたが、AutoFeatによる特徴量の自動生成で9個の特徴量が新たに追加され、結果的に39個の特徴量になりました。

number of features in X_train: 8
number of features in X_train_feature_creation: 57


    この自動生成した特徴量と同じ特徴量のテストデータを作ります。X_test_feature_creationに格納します。

In [ ]:
# モデル構築
model_1 = LinearRegression().fit(X_train,y_train)
model_2 = LinearRegression().fit(X_train_fsel, y_train)

    モデルの精度（R2：決定係数）を比較してみたいと思います。学習データとテストデータでそれぞれで比較してみます。

    先ず、学習データでモデルの精度（R2：決定係数）を比較します。

    以下、コードです。

In [ ]:
# 評価（R2）　※学習データ
print("model_1   R^2: %.4f" % r2_score(y_train, model_1.predict(X_train)) )
print("model_2   R^2: %.4f" % r2_score(y_train, model_2.predict(X_train_fsel)))

model_1   R^2: 0.5958
model_2   R^2: 0.6868
LassoLars R^2: 0.6686


    学習データを使った精度検証なので、基本高精度になります。

    元の特徴量のデータセットを使った方の決定係数は100%と高精度です。一方、特徴量を絞った新たな特徴量のデータセットを使った方の決定係数は99.71%と精度が若干落ちます。

    特徴量の数が減ったので当然の結果ですが、大幅に特徴量を減らした割に、それほど精度悪化していないことが分かります。

    次に、テストデータでモデルの精度（R2：決定係数）を比較します。

    以下、コードです。

In [ ]:
# 評価（R2）　※テストデータ
print("model_1   R^2: %.4f" % r2_score(y_test, model_1.predict(X_test)) )
print("model_2   R^2: %.4f" % r2_score(y_test, model_2.predict(X_test_fsel)))

# テストデータを使った精度検証なので、学習データに対する決定係数よりも基本悪化します。問題は、どの程度悪化するのか、ということになります

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11937 sha256=5b603467bdf12209067355a41e9127ca9d813959ff47e6d7997131499cd668b3
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit


    元の特徴量のデータセットを使った方の決定係数は49.39%と悪化します。
    一方、特徴量を絞った新たな特徴量のデータセットを使った方の決定係数は99.47%と高精度のままです。

    特徴量選択が上手くいっていることが分かるかと思います。

## 分類問題に対する特徴量選択例
    先ず、サンプルデータセットを生成します。

    以下、コードです。

In [ ]:
# サンプルデータの生成
X, y = make_classification(n_samples=1100,   #サンプルの数
                           n_features=1000,  #特徴量の数
                           n_informative=10, #目的変数と関連性の強い特徴量の数
                           n_classes=2,      #目的変数のクラスの数
                           random_state=12)

X = pd.DataFrame(X)
y = pd.DataFrame(y)

    生成したデータセットを学習データとテストデータに分割します。

    以下、コードです。

In [ ]:
# データセットの分割（学習データとテストデータ）
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    train_size=0.5,
                                                    test_size=0.5,
                                                    random_state=42)

    学習データ
    特徴量：X_train
    目的変数：y_train
    
    テストデータ
    特徴量：X_test
    目的変数：y_test

    変数選択用のモデルを定義します。

    以下、コードです。

In [ ]:
# モデル定義
model = FeatureSelector(problem_type="classification",verbose=1)

    学習データを利用し特徴量選択を実施します。

    以下、コードです。

In [ ]:
# 特徴量選択（学習データ利用）
X_train_fsel = model.fit_transform(X_train, y_train)

    元の特徴量からどのくらい減ったのかを見てみます。

    以下、コードです。

In [ ]:
print("number of features in X_train:",  #元の特徴量Xの数
      X_train.shape[1])
print("number of features in X_train_fsel:",
      X_train_fsel.shape[1])             #選択された特徴量Xの数

    1,000個あった特徴量が、13個に減っています。

    これで、モデルの予測精度が恐ろしく悪化したのでは、身も蓋もありません。

    テストデータで確認してみます。

    この自動選択した特徴量と同じ特徴量のテストデータを作ります。X_test_fselに格納します。

    以下、コードです。

In [ ]:
# テストデータの特徴量生成
X_test_fsel = model.transform(X_test)

    元の特徴量のデータセット（X_trainとy_train）と新たな特徴量のデータセット（X_train_fselとy_train）で、ロジスティック回帰モデルを構築してみます。

    以下、コードです。

In [ ]:
# モデル構築
## model_1
model_1 = LogisticRegression(class_weight='balanced')
model_1.fit(X_train,y_train)

## model_2
model_2 = LogisticRegression(class_weight='balanced')
model_2.fit(X_train_fsel, y_train)

    モデルの精度（accuracy：正答率）を比較してみたいと思います。
    学習データとテストデータでそれぞれで比較してみます。

    先ず、学習データでモデルの精度（accuracy：正答率）を比較します。

    以下、コードです。



In [ ]:
# 評価（accuracy）　※学習データ
print("model_1 accuracy:%.4f" % accuracy_score(y_train,
      model_1.predict(X_train)) )
print("model_2 accuracy:%.4f" % accuracy_score(y_train,
      model_2.predict(X_train_fsel)))

    学習データを使った精度検証なので、基本高精度になります。

    元の特徴量のデータセットを使った方の正答率は100%と高精度です。一方、特徴量を絞った新たな特徴量のデータセットを使った方の正答率は88.55%と精度が若干落ちます。

    特徴量の数が減ったので当然の結果ですが、大幅に特徴量を減らした割に、それほど精度悪化していないことが分かります。

    次に、テストデータでモデルの精度（accuracy：正答率）を比較します。

    以下、コードです。

In [ ]:
# 評価（accuracy）　※テストデータ
print("model_1 accuracy:%.4f" % accuracy_score(y_test,
      model_1.predict(X_test)) )
print("model_2 accuracy:%.4f" % accuracy_score(y_test,
      model_2.predict(X_test_fsel)))

    テストデータを使った精度検証なので、学習データに対する正答率よりも基本悪化します。問題は、どの程度悪化するのか、ということになります。

    元の特徴量のデータセットを使った方の正答率は75.64%と悪化します。一方、特徴量を絞った新たな特徴量のデータセットを使った方の正答率は81.64%と高精度のままです。

    特徴量選択が上手くいっていることが分かるかと思います。

## まとめ
    人によっては、特徴量選択の機能だけ使いたい、という方もいるかもしれません。今回は、特徴量選択の機能だけ使う場合のやり方について、簡単に説明しました。